In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import TomekLinks
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#Import other necessary model libraries, for this example, using Logistic Regression

## Import dataset

In [2]:
train_data = pd.read_csv('../data/train_dataset_v2.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10500 non-null  int64  
 1   device_type                  10477 non-null  object 
 2   device_category              10470 non-null  object 
 3   gender                       10365 non-null  object 
 4   district_name                10500 non-null  object 
 5   age_group                    10365 non-null  object 
 6   network_stay                 10500 non-null  int64  
 7   average_monthly_bill_amount  10500 non-null  float64
 8   dusage_sum                   10500 non-null  int64  
 9   dusage_min                   10500 non-null  float64
 10  dusage_max                   10500 non-null  float64
 11  dusage_diff                  10500 non-null  float64
 12  dusage_avg                   9642 non-null   float64
 13  dusage_days     

In [4]:
train_data.columns

Index(['primary_identifier', 'device_type', 'device_category', 'gender',
       'district_name', 'age_group', 'network_stay',
       'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max',
       'dusage_diff', 'dusage_avg', 'dusage_days', 'dusage_stddev',
       'vusage_onnet_sum', 'vusage_onnet_max', 'vusage_onnet_min',
       'vusage_onnet_diff', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_diff', 'vusage_offnet_days',
       'vusage_offnet_avg', 'vusage_offnet_stddev',
       'number_of_fixed_bb_accounts', 'number_of_iptv_accounts',
       'add_on_tot_rental', 'add_on_count', 'next_month_plan'],
      dtype='object')

In [5]:
# cols_remove = ['dusage_sum', 'dusage_days', 'dusage_stddev', 'dusage_min','dusage_max',
#                'vusage_onnet_sum','vusage_onnet_days','vusage_onnet_stddev', 'vusage_onnet_max',
#                'vusage_onnet_min','vusage_offnet_sum', 'vusage_offnet_days',
#                'vusage_offnet_stddev', 'vusage_offnet_max', 'vusage_offnet_min']

In [6]:
# cols_remove = ['dusage_sum', 'dusage_days', 'dusage_min','dusage_max',
#                'vusage_onnet_sum','vusage_onnet_days','vusage_onnet_max',
#                'vusage_onnet_min','vusage_offnet_sum', 'vusage_offnet_days',
#                'vusage_offnet_max', 'vusage_offnet_min']

In [7]:
# cols_remove = ['dusage_min','dusage_max', 'vusage_onnet_max', 'vusage_onnet_min',
#                'vusage_offnet_max', 'vusage_offnet_min', 'network_stay']

In [8]:
cols_remove = []

### Encoding Class Labels

In [9]:
train_data['next_month_plan'] = train_data['next_month_plan'].astype('category')

In [10]:
train_data['next_month_plan'].unique()

['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']
Categories (8, object): ['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']

In [11]:
labels_x = ['PKG0','PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7', 'PKG8']

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
le.fit(labels_x)

LabelEncoder()

In [14]:
train_data['encoded_class_labels'] = le.transform(train_data['next_month_plan'])

In [15]:
train_data = train_data.drop(columns=['next_month_plan', 'primary_identifier'])

In [16]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  10477 non-null  object 
 1   device_category              10470 non-null  object 
 2   gender                       10365 non-null  object 
 3   district_name                10500 non-null  object 
 4   age_group                    10365 non-null  object 
 5   network_stay                 10500 non-null  int64  
 6   average_monthly_bill_amount  10500 non-null  float64
 7   dusage_sum                   10500 non-null  int64  
 8   dusage_min                   10500 non-null  float64
 9   dusage_max                   10500 non-null  float64
 10  dusage_diff                  10500 non-null  float64
 11  dusage_avg                   9642 non-null   float64
 12  dusage_days                  10500 non-null  int64  
 13  dusage_stddev   

## Handling missing values

In [17]:
train_data['device_type'] = train_data['device_type'].fillna(value = 'Unknown')
train_data['device_category'] = train_data['device_category'].fillna(value = 'Unknown')
train_data['gender'] = train_data['gender'].fillna(value = 'Unknown')
train_data['age_group'] = train_data['age_group'].fillna(value = 'Unknown')

In [18]:
train_data['dusage_avg'] = train_data['dusage_avg'].fillna(value = 0)
train_data['vusage_offnet_avg'] = train_data['vusage_offnet_avg'].fillna(value = 0)
train_data['add_on_tot_rental'] = train_data['add_on_tot_rental'].fillna(value = -100)
train_data['add_on_count'] = train_data['add_on_count'].fillna(value = 0)
train_data['vusage_onnet_avg'] = train_data['vusage_onnet_avg'].fillna(value = 0)

In [19]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  10500 non-null  object 
 1   device_category              10500 non-null  object 
 2   gender                       10500 non-null  object 
 3   district_name                10500 non-null  object 
 4   age_group                    10500 non-null  object 
 5   network_stay                 10500 non-null  int64  
 6   average_monthly_bill_amount  10500 non-null  float64
 7   dusage_sum                   10500 non-null  int64  
 8   dusage_min                   10500 non-null  float64
 9   dusage_max                   10500 non-null  float64
 10  dusage_diff                  10500 non-null  float64
 11  dusage_avg                   10500 non-null  float64
 12  dusage_days                  10500 non-null  int64  
 13  dusage_stddev   

## Pre-processing Data

In [20]:
train_data = train_data.drop(columns=cols_remove)

In [21]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  10500 non-null  object 
 1   device_category              10500 non-null  object 
 2   gender                       10500 non-null  object 
 3   district_name                10500 non-null  object 
 4   age_group                    10500 non-null  object 
 5   network_stay                 10500 non-null  int64  
 6   average_monthly_bill_amount  10500 non-null  float64
 7   dusage_diff                  10500 non-null  float64
 8   dusage_avg                   10500 non-null  float64
 9   vusage_onnet_diff            10500 non-null  float64
 10  vusage_onnet_avg             10500 non-null  float64
 11  vusage_offnet_diff           10500 non-null  float64
 12  vusage_offnet_avg            10500 non-null  float64
 13  number_of_fixed_

In [22]:
X = train_data.drop(columns=['encoded_class_labels'])

In [23]:
y = train_data['encoded_class_labels']

In [24]:
def get_col_types(dataframe):
    data_cat_cols = []
    data_quan_cols = []
    
    for col in np.array(dataframe.columns):
        if dataframe[col].dtype == 'int64' or dataframe[col].dtype == 'float64':
            data_quan_cols.append(col)
        elif dataframe[col].dtype == 'O':
            data_cat_cols.append(col)
            
    return data_cat_cols, data_quan_cols

In [25]:
data_cat_cols, data_quan_cols = get_col_types(X)

In [26]:
data_cat_cols

['device_type', 'device_category', 'gender', 'district_name', 'age_group']

In [27]:
data_quan_cols

['network_stay',
 'average_monthly_bill_amount',
 'dusage_diff',
 'dusage_avg',
 'vusage_onnet_diff',
 'vusage_onnet_avg',
 'vusage_offnet_diff',
 'vusage_offnet_avg',
 'number_of_fixed_bb_accounts',
 'number_of_iptv_accounts',
 'add_on_tot_rental',
 'add_on_count']

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=10, test_size = 0.05, stratify = y)

In [29]:
quan_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

quan_transformed = quan_pipeline.fit_transform(X_train[data_quan_cols])

In [30]:
data_pipeline = ColumnTransformer([
    ('numerical', quan_pipeline, data_quan_cols),
    ('categorical', OrdinalEncoder(), data_cat_cols),
    
])

train_data_processed = data_pipeline.fit_transform(X_train)

In [31]:
test_data_processed = data_pipeline.transform(X_test)

## Loading Submission Data

In [32]:
submission_data = pd.read_csv("../data/test_dataset_v2.csv")

In [33]:
submission_data['device_type'] = submission_data['device_type'].fillna(value = 'Unknown')
submission_data['device_category'] = submission_data['device_category'].fillna(value = 'Unknown')
submission_data['gender'] = submission_data['gender'].fillna(value = 'Unknown')
submission_data['age_group'] = submission_data['age_group'].fillna(value = 'Unknown')

In [34]:
submission_data['dusage_avg'] = submission_data['dusage_avg'].fillna(value = 0)
submission_data['vusage_offnet_avg'] = submission_data['vusage_offnet_avg'].fillna(value = 0)
submission_data['add_on_tot_rental'] = submission_data['add_on_tot_rental'].fillna(value = -100)
submission_data['add_on_count'] = submission_data['add_on_count'].fillna(value = 0)
submission_data['vusage_onnet_avg'] = submission_data['vusage_onnet_avg'].fillna(value = 0)

In [35]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           4500 non-null   int64  
 1   device_type                  4500 non-null   object 
 2   device_category              4500 non-null   object 
 3   gender                       4500 non-null   object 
 4   district_name                4500 non-null   object 
 5   age_group                    4500 non-null   object 
 6   network_stay                 4500 non-null   int64  
 7   average_monthly_bill_amount  4500 non-null   float64
 8   dusage_sum                   4500 non-null   int64  
 9   dusage_min                   4500 non-null   float64
 10  dusage_max                   4500 non-null   float64
 11  dusage_diff                  4500 non-null   float64
 12  dusage_avg                   4500 non-null   float64
 13  dusage_days       

In [36]:
prim_id = submission_data['primary_identifier']

In [37]:
submission_X = submission_data.drop(columns=['primary_identifier'])
submission_X = submission_X.drop(columns=cols_remove)

In [38]:
submission_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  4500 non-null   object 
 1   device_category              4500 non-null   object 
 2   gender                       4500 non-null   object 
 3   district_name                4500 non-null   object 
 4   age_group                    4500 non-null   object 
 5   network_stay                 4500 non-null   int64  
 6   average_monthly_bill_amount  4500 non-null   float64
 7   dusage_diff                  4500 non-null   float64
 8   dusage_avg                   4500 non-null   float64
 9   vusage_onnet_diff            4500 non-null   float64
 10  vusage_onnet_avg             4500 non-null   float64
 11  vusage_offnet_diff           4500 non-null   float64
 12  vusage_offnet_avg            4500 non-null   float64
 13  number_of_fixed_bb

In [39]:
submission_X.shape

(4500, 17)

In [40]:
submission_data_processed = data_pipeline.transform(submission_X)

In [41]:
submission_data_processed[0].shape

(17,)

## Oversampling

In [42]:
unique, counts = np.unique(Y_train, return_counts=True)
dict(zip(unique, counts))

{1: 3490, 2: 1286, 3: 1631, 4: 698, 5: 1480, 6: 703, 7: 346, 8: 341}

In [43]:
os_size = 3500

In [44]:
strategy_os = {
    1: os_size,
    2: os_size,
    3: os_size,
    4: os_size,
    5: os_size,
    6: os_size,
    7: os_size,
    8: os_size
}

In [45]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42, sampling_strategy = strategy_os)
X_res, y_res = sm.fit_resample(train_data_processed, Y_train)

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3500) in class 1 will be larger than the number of samples in the majority class (class #1 -> 3490)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3500) in class 2 will be larger than the number of samples in the majority class (class #1 -> 3490)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3500) in class 3 will be larger than the number of samples in the majority class (class #1 -> 3490)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/im

In [46]:
unique, counts = np.unique(y_res, return_counts=True)
dict(zip(unique, counts))

{1: 3500, 2: 3500, 3: 3500, 4: 3500, 5: 3500, 6: 3500, 7: 3500, 8: 3500}

## Models

In [53]:
model_ran_for_new = RandomForestClassifier(random_state=0)

In [54]:
model_ran_for_new.fit(X_res, y_res)

RandomForestClassifier(random_state=0)

In [55]:
y_pred_ran_for_new = model_ran_for_new.predict(test_data_processed)

In [56]:
print(classification_report(Y_test, y_pred_ran_for_new))

              precision    recall  f1-score   support

           1       0.76      0.70      0.73       183
           2       0.39      0.32      0.35        68
           3       0.44      0.53      0.48        86
           4       0.29      0.32      0.31        37
           5       0.66      0.63      0.64        78
           6       0.42      0.38      0.40        37
           7       0.33      0.28      0.30        18
           8       0.31      0.56      0.40        18

    accuracy                           0.55       525
   macro avg       0.45      0.47      0.45       525
weighted avg       0.56      0.55      0.55       525



In [57]:
# for i in random_states_note:
#     print("The random state is ",i)
#     model = RandomForestClassifier(max_depth = 10, random_state = i, n_estimators = 100,
#                                    warm_start = True, bootstrap = False)
#     model.fit(X_bal, y_bal)
#     y_pred = model.predict(test_data_processed)
#     print(classification_report(Y_test, y_pred))

In [58]:
y_submission = model_ran_for_new.predict(submission_data_processed)

In [59]:
unique, counts = np.unique(y_submission, return_counts=True)
dict(zip(unique, counts))

{1: 1466, 2: 504, 3: 826, 4: 366, 5: 687, 6: 297, 7: 140, 8: 214}

In [157]:
model_xgb = XGBClassifier(random_state = 0, max_depth = 10)

In [158]:
model_xgb.fit(X_res, y_res)

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:32:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [159]:
y_pred_xgb = model_xgb.predict(test_data_processed)

In [160]:
print(classification_report(Y_test, y_pred_xgb))

              precision    recall  f1-score   support

           1       0.73      0.79      0.76       183
           2       0.38      0.25      0.30        68
           3       0.45      0.55      0.49        86
           4       0.25      0.16      0.20        37
           5       0.69      0.71      0.70        78
           6       0.42      0.38      0.40        37
           7       0.07      0.06      0.06        18
           8       0.38      0.56      0.45        18

    accuracy                           0.56       525
   macro avg       0.42      0.43      0.42       525
weighted avg       0.54      0.56      0.55       525



In [51]:
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.model_selection import HalvingGridSearchCV

In [52]:
#clf = RandomForestClassifier(random_state=0)

In [53]:
# param_grid = {"max_depth": [3, 5, 8, 11, 15, 20, 25],
#              "max_features": ['sqrt', 'log2', 20, 15]}

In [54]:
#search = HalvingGridSearchCV(clf, param_grid, resource='n_estimators',max_resources=400,random_state=0, cv=5)

In [55]:
#search.fit(X_res, y_res)

In [56]:
#search.best_params_

In [161]:
model_params = RandomForestClassifier(max_depth=10, n_estimators = 300, random_state=0)

In [162]:
model_params.fit(X_res, y_res)

RandomForestClassifier(max_depth=10, n_estimators=300, random_state=0)

In [163]:
y_pred_param = model_params.predict(test_data_processed)

In [164]:
print(classification_report(Y_test, y_pred_param))

              precision    recall  f1-score   support

           1       0.79      0.64      0.71       183
           2       0.32      0.31      0.32        68
           3       0.41      0.40      0.40        86
           4       0.28      0.57      0.37        37
           5       0.64      0.60      0.62        78
           6       0.43      0.32      0.37        37
           7       0.14      0.11      0.12        18
           8       0.38      0.78      0.51        18

    accuracy                           0.51       525
   macro avg       0.42      0.47      0.43       525
weighted avg       0.55      0.51      0.52       525



In [165]:
from sklearn.multiclass import OneVsOneClassifier

In [166]:
clf = OneVsOneClassifier(estimator=RandomForestClassifier(max_depth=10, n_estimators = 300, random_state=0),
                        n_jobs = -1)

In [167]:
clf.fit(X_res, y_res)

OneVsOneClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                    n_estimators=300,
                                                    random_state=0),
                   n_jobs=-1)

In [168]:
y_pred_clf = clf.predict(test_data_processed)

In [169]:
print(classification_report(Y_test, y_pred_clf))

              precision    recall  f1-score   support

           1       0.78      0.65      0.71       183
           2       0.35      0.29      0.32        68
           3       0.45      0.52      0.48        86
           4       0.32      0.51      0.39        37
           5       0.76      0.68      0.72        78
           6       0.38      0.35      0.37        37
           7       0.12      0.11      0.12        18
           8       0.37      0.72      0.49        18

    accuracy                           0.54       525
   macro avg       0.44      0.48      0.45       525
weighted avg       0.57      0.54      0.55       525



In [96]:
# model_to_set = OneVsOneClassifier(RandomForestClassifier(random_state=0), n_jobs = -1)

In [97]:
# parameters = {
#     "estimator__max_depth": [4, 8, 12, 16, 20],
#     "estimator__n_estimators": [200, 300, 400, 500]
# }

In [98]:
# from sklearn.metrics import f1_score
# model_tunning = GridSearchCV(model_to_set, param_grid=parameters, scoring=f1_score, verbose=1)

In [99]:
#model_tunning.fit(X_res, y_res)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=0        1
1        6
2        6
3        1
4        8
        ..
25195    8
25196    8
25197    8
25198    8
25199    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=5923     1
5929     1
5931     1
5933     1
5934     1
        ..
27295    8
27296    8
27297    8
27298    8
27299    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=1982     1
1984     1
1987     1
1993     1
1996     1
        ..
25895    8
25896    8
25897    8
25898    8
25899    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=7874     1
7875     1
7878     1
7879     1
7880     1
        ..
27995    8
27996    8
27997    8
27998    8
27999    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

...

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=0        1
1        6
2        6
3        1
4        8
        ..
25195    8
25196    8
25197    8
25198    8
25199    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=5923     1
5929     1
5931     1
5933     1
5934     1
        ..
27295    8
27296    8
27297    8
27298    8
27299    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=1982     1
1984     1
1987     1
1993     1
1996     1
        ..
25895    8
25896    8
25897    8
25898    8
25899    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=7874     1
7875     1
7878     1
7879     1
7880     1
        ..
27995    8
27996    8
27997    8
27998    8
27999    8
Name: encoded_class_labels, Length: 5600, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:687: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/utils/validation.py", line 74, in inner_

GridSearchCV(estimator=OneVsOneClassifier(estimator=RandomForestClassifier(random_state=0),
                                          n_jobs=-1),
             param_grid={'estimator__max_depth': [4, 8, 12, 16, 20],
                         'estimator__n_estimators': [200, 300, 400, 500]},
             scoring=<function f1_score at 0x7fe7288e99e0>, verbose=1)

In [101]:
# model_tunning.best_params_

{'estimator__max_depth': 4, 'estimator__n_estimators': 200}

In [170]:
param_ovo_model = OneVsOneClassifier(RandomForestClassifier(random_state=0, max_depth=10, n_estimators = 200), n_jobs = -1)

In [171]:
param_ovo_model.fit(X_res, y_res)

OneVsOneClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                    n_estimators=200,
                                                    random_state=0),
                   n_jobs=-1)

In [172]:
y_pred_ovo = param_ovo_model.predict(test_data_processed)

In [173]:
print(classification_report(Y_test, y_pred_ovo))

              precision    recall  f1-score   support

           1       0.78      0.65      0.71       183
           2       0.34      0.29      0.31        68
           3       0.46      0.56      0.50        86
           4       0.31      0.46      0.37        37
           5       0.74      0.67      0.70        78
           6       0.39      0.35      0.37        37
           7       0.12      0.11      0.12        18
           8       0.34      0.67      0.45        18

    accuracy                           0.54       525
   macro avg       0.44      0.47      0.44       525
weighted avg       0.57      0.54      0.55       525



In [174]:
clf_xgb = OneVsOneClassifier(estimator = XGBClassifier(max_depth = 10, random_state = 0))

In [175]:
clf_xgb.fit(X_res, y_res)

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


OneVsOneClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=10,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=0, reg_alpha=None,
                                           reg_lambda=No

In [176]:
y_pred_clf_xgb = clf_xgb.predict(test_data_processed)

In [177]:
print(classification_report(Y_test, y_pred_clf_xgb))

              precision    recall  f1-score   support

           1       0.72      0.75      0.74       183
           2       0.33      0.22      0.27        68
           3       0.45      0.52      0.48        86
           4       0.31      0.27      0.29        37
           5       0.65      0.71      0.68        78
           6       0.44      0.38      0.41        37
           7       0.23      0.17      0.19        18
           8       0.36      0.56      0.43        18

    accuracy                           0.55       525
   macro avg       0.44      0.45      0.44       525
weighted avg       0.54      0.55      0.54       525

